**If you lost points on the last checkpoint you can get them back by responding to TA/IA feedback**  

Update/change the relevant sections where you lost those points, make sure you respond on GitHub Issues to your TA/IA to call their attention to the changes you made here.

Please update your Timeline... no battle plan survives contact with the enemy, so make sure we understand how your plans have changed.

# COGS 108 - EDA Checkpoint

# Names

- Kavin Raj
- Arnav Saxena
- Tiantong Wu
- Peike Xu
- Jing Yin Yip

# Research Question

How do opinionated tweets on Twitter as measured by a sentiment analysis model affect the stock price of Tesla within the time period of September 2021 to September 2022?

## Background and Prior Work

Twitter is a widely used social media platform for expressing many kinds of opinions, including those on certain companies. We believe that this may have an impact on the public perception of these companies, and we aim to investigate whether there is a connection between opinions expressed in social media and the actual stock prices of the companies in question. An example of was seen this back in 2021, when “a thread on r/WallStreetBets”  caused “more than 7,200% increase in GME—and a 689% run”<a name="cite_note-1"></a> <sup>[<a href="#cite_ref-1">1</a>]</sup> . This occurence informed us that there is a potential causal effect between opinions on social media and real-world stock prices; we think that this effect has a much greater scope than just this isolated case of GameStop stocks, and we are interested in seeing if this is a larger, more general phenomenon that can be applied to other time periods and companies.

When reading a research paper from the IOP conference series, we found a sentiment analysis model based on social media opinion on stock trading. It was remarked in the conclusion that “looking into correlation coefficient compared by number of days before and after the trading day, the result shows that correlation reaches to the peak on trading day then it gradually declines with the magnitude depending on the day length after trading day.” <a name="cite_note-2"></a> <sup>[<a href="#cite_ref-2">2</a>]</sup> This research is similar to what we aim to investigate, as the paper conducted their test on a Thai social media platform called Pantip, and discussed ten Thai companies. The paper provides more evidence that there is a causal link between social media sentiment and stock prices, and we are interested to see if a similar trend can be seen with tweets and tech companies' stocks in the US. Additionally, we would also like to investigate if there are other similarities or differences in the trends that we are able to identify between our research and the paper, such as the correlation between variables reaching a peak on trading day.

1. <a name="cite_note-1"></a> <sup>[<a href="#cite_ref-1">1</a>]</sup> Rechel, J. (28 Jan 2021) How social media moves markets: Analyzing GameStop (GME) using social listening data. Sprout Blog.
   <br> https://sproutsocial.com/insights/gamestop-stock-social-media

2. <a name="cite_note-2"></a> <sup>[<a href="#cite_ref-2">2</a>]</sup>  P Padhanarath et al 2019 IOP Conf. Ser.: Mater. Sci. Eng. 620 012094.
   <br>https://iopscience.iop.org/article/10.1088/1757-899X/620/1/012094/pdf


# Hypothesis


Before answering our research question by investigating the available data, we think that there will be a positive correlation between a positive sentiment for a company and said company’s stock prices rising, as well as a positive correlation between a negative sentiment for a company and said company’s stock prices falling. This is due to the connection between a company’s public perception, how that is reflected in social media, and how it manifests in the stock market. If tweets about a company are mostly negative within a certain time period, we would expect to observe a decrease in stock prices, as both phenomena correspond to a decrease in public perception of the company. However, we also acknowledge that this relationship may not be as straightforward as is stated here, as there may be other confounds affecting each variable, such as Twitter only capturing the sentiment of a more vocal sample of people as compared to the rest of the population.

# Data

## Data overview

- Dataset #1
  - Dataset Name: Tweet Stock
  - Link to the dataset: https://www.kaggle.com/code/shreytandel19/stock-prediction-based-on-tweet-sentiment-analysis/input
  - Number of observations: 80793
  - Number of variables: 4
- Dataset #2
  - Dataset Name: TESLA_HISTORICAL
  - Link to the dataset: https://www.nasdaq.com/market-activity/stocks/tsla/historical
  - Number of observations: 1258
  - Number of variables: 6

The first dataset, Tweet Stock, is a collection of tweets that talk about TSLA, the stock for the company Tesla. The time range for this datasets is from September 30th, 2021 to September 29th, 2022, and was chosen  since we are unable to obtain current Twitter data, and need to rely on past data that has already been collected. The two important variables in this dataset are the dates that the tweets were posted, as well as the tweets themselves. The tweets are in the form of strings, and the dates will be converted into a standard Timestamp format. The tweets in this dataset will be used as a proxy for social media opinion, once it is processed by a sentiment analysis model.

The second dataset, TESLA_HISTORICAL, is a record of prices of the TSLA stock on each day from September 30th, 2021 to September 29th, 2022. This time range was chosen to match the time range of the other dataset. In the dataset, the highest, lowest, and open/close prices of the stock are recorded as floats in each row, along with the corresponding day, which will be converted into a standard Timestamp format. These variables can provide insight into how a stock varies within a day, and comparing these across different days can lead to potential trends over time. The prices are a direct factual record of TSLA prices, although further numerical processing may be needed later to reveal meaningful trends.

Further details of the specific steps taken to clean the datasets will be explained below.

## Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns

### Tweet Stock

We read the csv file, convert it to a dataframe, and drop any rows that contain empty values. Then, dates in the "Date" variable are converted  into standard Timestamp format. Afterwards, we narrow down the tweets to anything that includes "TSLA", and convert all the tweets into lower case, which is needed later for sentiment analysis.

In [2]:
tweet = pd.read_csv('stock_tweets.csv')
tweet.dropna(inplace=True)
#delete any missing value within the dataset
tweet['Date'] = pd.to_datetime(tweet.get('Date')).dt.date
#convert the date enteries into standarized form
tweet_tesla = tweet[tweet.get('Stock Name')=='TSLA']
#reduced the dataset into Tesla only
tweet_tesla['Tweet'] = tweet_tesla['Tweet'].str.lower()
#convert all the tweet into lower case for easier analysis
# %pip install nltk
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# tweet_tesla['Tweet'] = tweet_tesla['Tweet'].apply(word_tokenize)
# stop_words = set(stopwords.words('english'))
# tweet_tesla['Tweet'].apply(lambda x: [item for item in x if item not in stop_words])
# #tokenize the tweet and remove stopwords for furthur sentiment analysis
tweet_tesla

/var/folders/j0/5bxbmj_d11v9ht5xvzjhn4p00000gn/T/ipykernel_3257/4288841202.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_tesla['Tweet'] = tweet_tesla['Tweet'].str.lower()


,Date,Tweet,Stock Name,Company Name
0,2022-09-29,mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29,tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29,3/ even if i include 63.0m unvested rsus as of...,TSLA,"Tesla, Inc."
3,2022-09-29,@realdanodowd @wholemarsblog @tesla hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29,"@realdanodowd @tesla stop trying to kill kids,...",TSLA,"Tesla, Inc."
...,...,...,...,...
37417,2021-09-30,playing in the dirt and #chasingsunsets\n@tesl...,TSLA,"Tesla, Inc."
37418,2021-09-30,i agree with @freshjiva that $tsla ‘s ev busin...,TSLA,"Tesla, Inc."
37419,2021-09-30,hold. on. tight. $tsla,TSLA,"Tesla, Inc."
37420,2021-09-30,get ready for a $tsla _ _ _ _ _ _ q3 delivery...,TSLA,"Tesla, Inc."


## TESLA_HISTORICAL

We first read the csv file for Tesla and convert it into a dataframe. Following that, any rows that contain empty values are dropped. We then convert the date to standard Timestamp format, to make it easier to manipulate the certain dates we are looking for. For the price variables, dollar signs in the entries are removed, and the numerical values are converted to floats. Lastly, the appropriate time range is selected to match our dataset of Tweets.

In [4]:
tesla = pd.read_csv('TESLA_HISTORICAL.csv')
tesla.dropna(inplace=True)
#delete any missing value within the dataset
tesla['Date'] = pd.to_datetime(tesla.get('Date')).dt.date
#convert the date enteries into standarized form
def nodollartofloat(series):
    series = series.str.strip('$')
    blank = []
    for i in series:
        i = float(i)
        blank = np.append(blank,i)
    return blank
tesla['Close/Last'] = nodollartofloat(tesla['Close/Last'])
tesla['Open'] = nodollartofloat(tesla['Open'])
tesla['High'] = nodollartofloat(tesla['High'])
tesla['Low'] = nodollartofloat(tesla['Low'])
#convert all string input into number

start_date = pd.to_datetime('2021-09-30').date()
end_date = pd.to_datetime('2022-09-29').date()

tesla = tesla[(tesla['Date'] >= start_date) & (tesla['Date'] <= end_date)]
tesla.reset_index(drop=True, inplace=True)
tesla

,Date,Close/Last,Volume,Open,High,Low
0,2022-09-29,268.2100,77620640,282.7600,283.6500,265.7800
1,2022-09-28,287.8100,54664810,283.0800,289.0000,277.5700
2,2022-09-27,282.9400,61925190,283.8400,288.6700,277.5100
3,2022-09-26,276.0100,58076910,271.8300,284.0900,270.3100
4,2022-09-23,275.3300,63748360,283.0900,284.5000,272.8200
...,...,...,...,...,...,...
247,2021-10-06,260.9166,43898314,258.7333,262.2200,257.7400
248,2021-10-05,260.1966,55297896,261.5986,265.7700,258.0666
249,2021-10-04,260.5100,91450029,265.5000,268.9899,258.7066
250,2021-10-01,258.4066,51094235,259.4666,260.2600,254.5300


# Results

## Exploratory Data Analysis

Carry out whatever EDA you need to for your project.  Because every project will be different we can't really give you much of a template at this point. But please make sure you describe the what and why in text here as well as providing interpretation of results and context.

### Section 1 of EDA - please give it a better title than this

Some more words and stuff.  Remember notebooks work best if you interleave the code that generates a result with properly annotate figures and text that puts these results into context.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

### Section 2 of EDA if you need it  - please give it a better title than this

Some more words and stuff.  Remember notebooks work best if you interleave the code that generates a result with properly annotate figures and text that puts these results into context.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

In our project, there exist several ethical and privacy concerns. First of all, to collect users' twitter posts might be a violation of privacy. Although Twitter posts are publicly posted, people might not want to have their posts be a part of a project or an experiment. Thus, we planned to handle this privacy issue by anonymizing all data from the users. In addition, there exists a potential bias that it is hard to judge a human written post as simply positive or negative. Some posts might contain emotions that are complex. People might be confused about the trend of a stock's price, or hard to determine whether they are clearly for or clearly against a change in the stock prices. Also, we should be able to accurately interpret the information contained in a twitter post. To handle this issue, we must present our findings as transparently as possible. Also, since our data set is relatively large(there are thousands of twitter posts), we could clearly identify the ones who are less complex or have clear stances, and analyze the correlation between those posts and the stock prices.

One of the concerns is that our data relies heavily on the judgements of a sentiment analysis model. It's important to use these models with caution, be aware of their potential shortcomings, and complement them with human judgment and context whenever possible to ensure the most accurate and reliable analysis of sentiment in your data. Also, the words defined for negative in a stock context might be different with general sentiment analysis. We can try out several analysis models and compare the results between different models, finding the most accurate model. 

# Team Expectations 

- **Timely Project Work**:

    Punctuality: Complete tasks promptly, meet deadlines, and maintain a consistent effort for synchronized project progress.

    Equitable Contribution: Ensure a fair distribution of responsibilities by contributing proportionately to the project workload.

    Proactivity: Embrace a proactive work ethic by actively engaging with project requirements.

- **Open and Consistent Communication**:

    Transparency: Share information openly about task progress, challenges, and potential workloads.

    Conflict Resolution: Address conflicts promptly, seeking resolutions or proposing solutions for effective problem-solving.

    Regular Updates: Provide consistent updates on task status, checkpoints, and any changes to the project plan.

<!-- - Keeping up with the project work in a timely manner with everyone providing similar effort to the project
- Having open and consistent communication to update everyone on any conflicts and issues in order to proceed with the project smoothly.  -->

# Project Timeline Proposal

Two aspects that we may need guidance on are: obtaining data from social media sites where APIs may not be available; and using sentiment analysis models to determine whether text is positve or negative towards certain entities.



| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 11/5  |  1 PM | Proposal  | Collect Datasets, Assign EDA Tasks | 
| 11/12  |  1 PM |  Collection of Raw Data; Data cleaning | Completing Checkpoint 1 | 
| 11/20  | 1 PM | Seeking more datas; Starting setiment analysis | Look for datasets of other potential stocks/tweets; Start EDA for existing data |
| 11/25  | 1 PM | Finishing up EDA, starting analysis | Completing Checkpoint 2 |
| 12/3  | 1 PM  | Draft the report, discussion results | Start Final Project Report |
| 12/11 | 1 PM  | Finalize Project |Prepare for the final presentation |